In [1]:
# Declaring all modules and libraries to be used in the project
import pandas as pd
import requests
import json
import time
import numpy as np
import copy
import sys, os, re
import random
from functools import partial
from multiprocessing import cpu_count
from multiprocessing.dummy import Pool as ThreadPool
from tqdm.notebook import tqdm
import function as fn
from datetime import datetime
from workalendar.oceania.australia import Australia
import ipywidgets
from fuzzywuzzy import fuzz
from concurrent.futures import ThreadPoolExecutor, as_completed
import threading
import ast



prefix = "price_to_hds_hand_unload_cario_"
DIMENSIONS = [90, 60, 120, 125, 1] #[L, W, H, Weight, Q]
exclusion = [""]

request_rates = True
id_request = False




/Users/justinng/opt/miniconda3/envs/shipping/lib/python3.10/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
# %% ▶️  Clean, cache-driven suburb → LocationID loader  (CSV edition)
import ast, json, re
from pathlib import Path

import pandas as pd
from rapidfuzz import process, fuzz        # pip install rapidfuzz

# ------------------------------------------------------------------ helpers
STATE_RE = re.compile(r'\b(?:NSW|QLD|VIC|WA|TAS|SA|ACT|NT)\b', flags=re.I)

def _clean_suburb(txt: str) -> str:
    """Uniform suburb string: remove state codes/digits, squeeze spaces, upper-case."""
    txt = STATE_RE.sub('', str(txt))
    txt = re.sub(r'\d+', '', txt)
    txt = re.sub(r'\s+', ' ', txt).strip()
    return txt.upper()

def _build_loc_df(location_data: dict) -> pd.DataFrame:
    rows = []
    for pc, locs in location_data.items():
        pc4 = str(pc).zfill(4)
        for loc in locs or []:
            sub = loc.get('suburb') or loc.get('description', '')
            rows.append(
                {
                    'postcode': pc4,
                    'suburb_clean': _clean_suburb(sub),
                    'location_id': loc['id'],
                }
            )
    return pd.DataFrame(rows).drop_duplicates()

def assign_ids(df, pc_col, sub_col, out_col, choices_df, fuzz_thresh=90):
    """Attach Cario location IDs with exact merge, then fuzzy fallback."""
    df = df.copy()
    df['pc4']       = df[pc_col].astype(str).str.zfill(4)
    df['sub_clean'] = df[sub_col].map(_clean_suburb)

    # exact join -------------------------------------------------------------
    df = df.merge(
        choices_df,
        left_on=['pc4', 'sub_clean'],
        right_on=['postcode', 'suburb_clean'],
        how='left',
    )
    df[out_col] = df['location_id']

    # fuzzy fallback ---------------------------------------------------------
    misses = df[df[out_col].isna()].index
    for pc4, idx in df.loc[misses].groupby('pc4').groups.items():
        cand = choices_df.loc[choices_df.postcode == pc4, 'suburb_clean']
        if cand.empty:
            continue
        choices = cand.tolist()
        for i in idx:
            target = df.at[i, 'sub_clean']
            best, score, _ = process.extractOne(target, choices, scorer=fuzz.WRatio)
            if score >= fuzz_thresh:
                df.at[i, out_col] = choices_df.query(
                    'postcode == @pc4 & suburb_clean == @best'
                ).iloc[0]['location_id']

    # Convert to nullable integer type to handle NaN properly
    df[out_col] = df[out_col].astype('Int64')  # Capital I for nullable int
    
    return df.drop(columns=['pc4', 'sub_clean', 'postcode', 'suburb_clean', 'location_id'])

# ------------------------------------------------------------------ 1) load cached location data (CSV)
csv_path = Path('data/cario_location_data.csv')
if not csv_path.exists():
    raise FileNotFoundError('Expected CSV cache at data/cario_location_data.csv – run the scraper first.')

location_data = {}
for _, row in pd.read_csv(csv_path).iterrows():
    try:
        location_data[str(row['Postcode']).zfill(4)] = ast.literal_eval(row['Response'])
    except Exception:
        continue  # skip unparsable rows

loc_df = _build_loc_df(location_data)

# ------------------------------------------------------------------ 2) load & clean your input datasets
aus = (
    pd.read_csv('input/australian_postcodes_2025.csv')
    .loc[lambda d: d.chargezone.notna(), ['postcode', 'locality', 'state', 'long', 'lat']]
    .rename(
        columns={
            'postcode': 'Customer Postcode',
            'locality': 'Customer Locality',
            'state': 'Customer State',
            'long': 'Customer Long',
            'lat': 'Customer Lat',
        }
    )
)
aus['Customer Locality'] = aus['Customer Locality'].str.title()

wh = (
    pd.read_excel('input/parameters.xlsx', 'Warehouses')
    .query("`Sending Warehouse`.isin(['Brisbane','Melbourne','Sydney','Perth'])")
    .rename(
        columns={
            'Suburb':  'Warehouse Suburb',
            'Address': 'Warehouse Address',
            'Postcode':'Warehouse Postcode',
            'State':   'Warehouse State',
            'Long':    'Warehouse Long',
            'Lat':     'Warehouse Lat',
        }
    )
    .query("`Warehouse Suburb` != 'Welshpool'")
)

# ------------------------------------------------------------------ 3) map to/from IDs
aus = assign_ids(aus, 'Customer Postcode',  'Customer Locality',  'to_id',   loc_df)
wh  = assign_ids(wh,  'Warehouse Postcode', 'Warehouse Suburb',   'from_id', loc_df)

# ------------------------------------------------------------------ 4) build unique combinations
unique_combinations = (
    aus.merge(
        wh[['Warehouse Postcode', 'Warehouse Suburb', 'Warehouse State', 'from_id']],
        how='cross',
    )
    .query('to_id.notnull() & from_id.notnull()')  # both IDs must be present
    .drop_duplicates(subset=['Customer Postcode', 'Warehouse Postcode'])
    .reset_index(drop=True)
)


# Testing
unique_combinations = unique_combinations.sample(n=200, random_state=42)


# 

In [6]:
# %% Async HTTP/2 Cario multi-account rate fetcher (httpx + uvloop + orjson) - 8 retries
import asyncio
import uvloop; uvloop.install()
import httpx, orjson, pandas as pd
from datetime import datetime, timedelta
from tqdm import tqdm
from contextlib import AsyncExitStack
import os
import random

MAX_RETRIES = 8
SLEEP_BASE  = 0.5  # seconds

# ---------- supply your creds and routing ----------
# Example only - load from env or a secrets file in practice
SESSION_TOKEN = "eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJodHRwOi8vc2NoZW1hcy54bWxzb2FwLm9yZy93cy8yMDA1LzA1L2lkZW50aXR5L2NsYWltcy9uYW1laWRlbnRpZmllciI6Ijk5MDkiLCJodHRwOi8vc2NoZW1hcy54bWxzb2FwLm9yZy93cy8yMDA1LzA1L2lkZW50aXR5L2NsYWltcy9uYW1lIjoiWl9HcmlsbHNfQXVzdHJhbGlhIiwiaHR0cDovL3NjaGVtYXMueG1sc29hcC5vcmcvd3MvMjAwNS8wNS9pZGVudGl0eS9jbGFpbXMvZW1haWxhZGRyZXNzIjoianVzdGluLm5nQHpncmlsbHMuY29tLmF1IiwiQXNwTmV0LklkZW50aXR5LlNlY3VyaXR5U3RhbXAiOiJNNVBVUFVZSUhYSVBYQ0ZSNVBDVjJWWVNMRFVUNFQ2QSIsImh0dHA6Ly9zY2hlbWFzLm1pY3Jvc29mdC5jb20vd3MvMjAwOC8wNi9pZGVudGl0eS9jbGFpbXMvcm9sZSI6IkRlc3BhdGNoUGFyZW50QWNjdCIsImh0dHA6Ly93d3cuYXNwbmV0Ym9pbGVycGxhdGUuY29tL2lkZW50aXR5L2NsYWltcy90ZW5hbnRJZCI6IjkiLCJDdXN0b21lcklEIjoiNzExNSIsInN1YiI6Ijk5MDkiLCJqdGkiOiJmZWZlM2ZlYi02M2Q5LTRmMDctODYyMi01NDRmOGFlYzgyOWUiLCJpYXQiOjE3NTY3OTQ5ODgsIm5iZiI6MTc1Njc5NDk4OCwiZXhwIjoxNzU2ODgxMzg4LCJpc3MiOiJGTVMiLCJhdWQiOiJGTVMifQ.dcHOlYolnRIbIsrrTif0rbV4XpjSSwCmhZdWqt1b3_w"
CREDENTIALS = {
    # keys are arbitrary labels you choose
    "MEL": {"customer_id": 16071, "auth_token": SESSION_TOKEN, "tenant_id": ""},
    "SYD": {"customer_id": 16070, "auth_token": SESSION_TOKEN, "tenant_id": ""},
    "BNE": {"customer_id": 16072, "auth_token": SESSION_TOKEN, "tenant_id": ""},
}

# Map each origin suburb in your DataFrame to an account key above
WH_TO_ACCOUNT = {
    "Epping": "SYD",
    "Chipping Norton": "SYD",
    "Berrinba":  "BNE",
}

BASE_URL  = "https://integrate.cario.com.au/api/"
COUNTRY   = {"id": 36, "iso2":"AU", "iso3":"AUS", "name":"AUSTRALIA"}

def _headers(cred: dict) -> dict:
    return {
        "Authorization": f"Bearer {cred['auth_token']}",
        "CustomerId": str(cred["customer_id"]),
        "TenantId": cred.get("tenant_id", ""),
        "Content-Type": "application/json",
    }

# ---------- helpers ----------
def get_next_weekday(today: datetime) -> str:
    wd = today.weekday()
    if wd in (5, 6):  # Sat or Sun -> next Monday
        return (today + timedelta(days=(7 - wd))).date().isoformat()
    return today.date().isoformat()

def build_payload(row: pd.Series, dims: list, customer_id: int) -> dict:
    L, W, H, weight, qty = dims
    return {
        "customerId":   customer_id,
        "pickupDate":   get_next_weekday(datetime.now()),
        "pickupAddress": {
            "name": "Quote", "line1": "Quote",
            "location": {
                "id": int(row["from_id"]),
                "locality": row["Warehouse Suburb"],
                "state":    row["Warehouse State"],
                "postcode": str(row["Warehouse Postcode"]),
                "country":  COUNTRY
            }
        },
        "deliveryAddress": {
            "name": "Quote", "line1": "Quote",
            "location": {
                "id": int(row["to_id"]),
                "locality": row["Customer Locality"],
                "state":    row["Customer State"],
                "postcode": str(row["Customer Postcode"]),
                "country":  COUNTRY
            },
            "isResidential": True
        },
        "totalItems":  qty,
        "totalWeight": weight,
        "transportUnits": [{
            "transportUnitType": "Pallet",
            "quantity": qty,
            "length":   L,
            "width":    W,
            "height":   H,
            "weight":   weight,
            "volume":   (L * W * H) / 1_000_000
        }],
        "optionHandUnload": True
    }

async def fetch_quote(client: httpx.AsyncClient, customer_id: int, row: pd.Series,
                      dims: list, sem: asyncio.Semaphore, row_index: int) -> tuple[int, dict]:
    async with sem:
        payload = build_payload(row, dims, customer_id)
        content = orjson.dumps(payload)
        for attempt in range(MAX_RETRIES):
            try:
                r = await client.post("/Consignment/GetQuotes", content=content)
                if r.status_code == 200:
                    return row_index, r.json()
                # Respect simple rate limit hints
                if r.status_code in (429, 503):
                    retry_after = r.headers.get("Retry-After")
                    if retry_after:
                        try:
                            await asyncio.sleep(float(retry_after))
                        except Exception:
                            await asyncio.sleep(SLEEP_BASE * (attempt + 1))
                    else:
                        # jittered backoff
                        await asyncio.sleep(SLEEP_BASE * (attempt + 1) * random.uniform(0.8, 1.3))
                    continue
                if attempt == MAX_RETRIES - 1:
                    return row_index, {"error": r.status_code, "msg": r.text[:500]}
            except httpx.TimeoutException:
                if attempt == MAX_RETRIES - 1:
                    return row_index, {"error": "timeout"}
            except Exception as e:
                if attempt == MAX_RETRIES - 1:
                    return row_index, {"error": "exception", "msg": str(e)}
            await asyncio.sleep(SLEEP_BASE * (attempt + 1) * random.uniform(0.8, 1.3))

async def fetch_all_multi(df: pd.DataFrame, dims: list,
                          creds: dict, wh_to_account: dict,
                          conc_per_account: int = 16, req_timeout: float = 60.0) -> pd.DataFrame:
    df = df.copy()
    df["Response"] = ""

    # Build one client and one semaphore per account
    async with AsyncExitStack() as stack:
        clients = {}
        sems    = {}
        for key, cred in creds.items():
            clients[key] = await stack.enter_async_context(
                httpx.AsyncClient(
                    base_url=BASE_URL,
                    headers=_headers(cred),
                    http2=True,
                    limits=httpx.Limits(max_keepalive_connections=1, max_connections=conc_per_account),
                    timeout=httpx.Timeout(req_timeout),
                )
            )
            sems[key] = asyncio.Semaphore(conc_per_account)

        tasks = []

        for i, row in df.iterrows():
            wh_suburb = str(row["Warehouse Suburb"])
            acct_key  = wh_to_account.get(wh_suburb)
            if acct_key is None:
                # Skip or raise. Here we mark an error row.
                df.at[i, "Response"] = orjson.dumps({"error": "missing_account_mapping", "warehouse": wh_suburb}).decode()
                continue

            client      = clients[acct_key]
            sem         = sems[acct_key]
            customer_id = creds[acct_key]["customer_id"]

            tasks.append(asyncio.create_task(
                fetch_quote(client, customer_id, row, dims, sem, i)
            ))

        for fut in tqdm(asyncio.as_completed(tasks), total=len(tasks), desc="Fetching quotes"):
            idx, res = await fut
            df.at[idx, "Response"] = orjson.dumps(res).decode()

    return df

def price_quote_cario_multi(df: pd.DataFrame, dims: list,
                            creds: dict, wh_to_account: dict,
                            conc_per_account: int = 16) -> pd.DataFrame:
    try:
        import nest_asyncio; nest_asyncio.apply()
    except ImportError:
        pass
    return asyncio.get_event_loop().run_until_complete(
        fetch_all_multi(df, dims, creds, wh_to_account, conc_per_account=conc_per_account)
    )

# ---------- usage ----------
# example: route all lanes in your current DataFrame
def filter_eastern_states_with_postcode(df, include_pc=3168, n_others=20, random_state=42):
    # filter to VIC, NSW, QLD customers
    east = df[df['Customer State'].isin(['VIC','NSW','QLD'])]
    
    # choose postcodes: force include_pc + n_others random others from filtered set
    others = east.loc[east['Customer Postcode'] != include_pc, 'Customer Postcode'].drop_duplicates()
    selected = {include_pc} | set(others.sample(n=min(n_others, len(others)), random_state=random_state))
    
    # get all rows for selected postcodes
    relevant_combo = east[east['Customer Postcode'].isin(selected)].reset_index(drop=True)
    return relevant_combo

relevant_lanes = filter_eastern_states_with_postcode(unique_combinations)

# ---------- usage ----------
if request_rates:

    # unique_combinations = filter_eastern_states_with_postcode(unique_combinations)
    print(f"⏳ Fetching quotes for {len(unique_combinations)} lanes…")

    out = price_quote_cario_multi(relevant_lanes, DIMENSIONS, CREDENTIALS, WH_TO_ACCOUNT, conc_per_account=32)
    ts  = datetime.now().strftime("%Y%m%d-%H%M%S")
    fn = f"data/{prefix}{ts}.csv"
    out.to_csv(fn, index=False)
    print(f"✅ Saved to {fn}")
    good = out["Response"].apply(lambda x: "error" not in orjson.loads(x)).sum()
    print(f"  • Success: {good}/{len(out)}")


⏳ Fetching quotes for 200 lanes…


Fetching quotes: 100%|██████████| 22/22 [00:40<00:00,  1.82s/it]

✅ Saved to data/price_to_hds_hand_unload_cario_20250903-135503.csv
  • Success: 22/22


In [ ]:
import os, re
from datetime import datetime

def get_latest_input_file(prefix: str) -> tuple[str | None, str | None]:
    """
    Scan the `data/` folder for files named like
      {prefix}YYYYMMDD-HHMMSS.csv
    and return both the filename and the timestamp string.
    """
    latest_ts: datetime | None = None
    latest_file: str | None = None
    # Pre‐compile regex for speed
    pattern = re.compile(rf'^{re.escape(prefix)}(\d{{8}}-\d{{6}})\.csv$')

    for fname in os.listdir('data/'):
        m = pattern.match(fname)
        if not m:
            continue
        ts_str = m.group(1)
        try:
            ts = datetime.strptime(ts_str, '%Y%m%d-%H%M%S')
        except ValueError:
            continue
        if latest_ts is None or ts > latest_ts:
            latest_ts = ts
            latest_file = fname

    if latest_file is None:
        return None, None

    # Return both filename and the extracted timestamp
    return latest_file, latest_ts.strftime('%Y%m%d-%H%M%S')

file, timestamp = get_latest_input_file(prefix)
print(f"Latest file: {file}, Timestamp: {timestamp}")

# Get the latest file and its timestamp
latest_file, latest_ts = get_latest_input_file(prefix)

if latest_file:
    # Read it into a DataFrame
    data = pd.read_csv(os.path.join('data', latest_file))
    print(f"Loaded '{latest_file}' (timestamp: {latest_ts}) — shape: {data.shape}")
else:
    data = pd.DataFrame()
    print("No matching file found.")

In [ ]:

def extract_best(row: pd.Series, scrape_ts: str) -> pd.Series:
    """
    From row['Response'] (a JSON list of quotes), pick the cheapest one
    and return price, transit_days, freight_company & carrier_id.
    """
    try:
        routes = json.loads(row.get('Response','[]'))
    except json.JSONDecodeError:
        routes = []
    if not isinstance(routes, list) or not routes:
        return pd.Series({
            'cheapest_price':   None,
            'transit_time':     None,
            'freight_company':  None,
            'carrier_id':       None
        })

    # find the cheapest by total
    best = min(routes, key=lambda r: r.get('total', float('inf')))
    price   = best.get('total')
    company = best.get('carrierName')
    cid     = best.get('carrierId')

    # compute transit days
    eta = best.get('eta')
    if eta:
        try:
            eta_dt   = datetime.fromisoformat(eta)
            base_dt  = datetime.strptime(scrape_ts, '%Y%m%d-%H%M%S')
            days     = (eta_dt - base_dt).days
        except Exception:
            days = None
    else:
        days = None

    return pd.Series({
        'cheapest_price':   price,
        'transit_time':     days,
        'freight_company':  company,
        'carrier_id':       cid
    })

# extract cheapest & transit
out = data.join(
    data.apply(lambda r: extract_best(r, latest_ts), axis=1)
)

out.head(2)



In [ ]:
data

In [ ]:
import pandas as pd
import json
from datetime import datetime

def get_cheapest_per_postcode_pair(data: pd.DataFrame, scrape_ts: str) -> pd.DataFrame:
    """
    Extracts the absolute cheapest freight rate for each unique 
    (Customer Postcode, Warehouse Postcode) pair.

    Applies special rule: If freight_company is 'Jet Couriers' and transit time is missing,
    defaults transit_time_days to 1.

    Returns DataFrame with standardized columns.
    """
    results = []
    
    for _, row in data.iterrows():
        # Parse quotes from Response
        try:
            quotes = json.loads(row['Response'])
        except (json.JSONDecodeError, TypeError):
            continue
            
        if not isinstance(quotes, list) or not quotes:
            continue
            
        # Find cheapest quote
        best_quote = min(quotes, key=lambda q: q.get('total', float('inf')))
        
        # Extract fields
        price = best_quote.get('total')
        carrier_id = best_quote.get('carrierId')
        company = best_quote.get('carrierName')
        
        # Calculate transit time
        transit_days = None
        if eta := best_quote.get('eta'):
            try:
                eta_dt = datetime.fromisoformat(eta)
                base_dt = datetime.strptime(scrape_ts, '%Y%m%d-%H%M%S')
                transit_days = (eta_dt - base_dt).days
            except Exception:
                pass  # leave as None if parsing fails
        
        # Special rule: Jet Couriers → default to 1-day transit if missing
        if company == 'Jet Couriers' and transit_days is None:
            transit_days = 1
        
        # Collect result
        results.append({
            'customer_postcode': row['Customer Postcode'],
            'customer_locality': row['Customer Locality'],
            'warehouse_postcode': row['Warehouse Postcode'],
            'warehouse_locality': row['Warehouse Suburb'],
            'freight_price': price,
            'transit_time_days': transit_days,
            'carrier_id': carrier_id,
            'freight_company': company
        })
    
    # Create DataFrame
    df = pd.DataFrame(results)
    if df.empty:
        return pd.DataFrame(columns=[
            'customer_postcode', 'customer_locality', 'warehouse_postcode',
            'warehouse_locality', 'freight_price', 'transit_time_days',
            'carrier_id', 'freight_company'
        ])
    
    # Sort by price so cheapest is first within each group
    df = df.sort_values('freight_price')
    
    # Keep only the cheapest quote per (customer_postcode, warehouse_postcode)
    final = df.drop_duplicates(
        subset=['customer_postcode', 'warehouse_postcode'],
        keep='first'
    ).reset_index(drop=True)
    
    # Convert numeric fields
    final['freight_price'] = pd.to_numeric(final['freight_price'], errors='coerce')
    final['transit_time_days'] = pd.to_numeric(final['transit_time_days'], errors='coerce')
    final['carrier_id'] = pd.to_numeric(final['carrier_id'], errors='coerce')
    
    # Final column order
    final = final[[
        'customer_postcode',
        'customer_locality',
        'warehouse_postcode',
        'warehouse_locality',
        'freight_price',
        'transit_time_days',
        'carrier_id',
        'freight_company'
    ]]
    
    return final

# Usage example:
cheapest_price_lane = get_cheapest_per_postcode_pair(data, latest_ts)

# Get Cheapest for Each Postcode
def get_cheapest_per_postcode(df):
    """
    Get the cheapest freight rate for each customer postcode across all warehouses.
    
    Args:
        df: DataFrame with columns including customer_postcode, freight_price
    
    Returns:
        DataFrame with cheapest rate per postcode
    """
    if df.empty:
        return df
    
    # Group by customer postcode and find the row with minimum freight price
    cheapest_per_postcode = df.loc[df.groupby('customer_postcode')['freight_price'].idxmin()].reset_index(drop=True)
    
    return cheapest_per_postcode

# Get cheapest rate for each customer postcode
cheapest_per_postcode = get_cheapest_per_postcode(cheapest_price_lane)
print(f"Cheapest rates for {len(cheapest_per_postcode)} unique postcodes:")
cheapest_per_postcode




# %%

In [ ]:
cheapest_price_lane[cheapest_price_lane['customer_postcode'] == 3168]

In [ ]:
def plot_hds_rates_map(hds_data, warehouse_data=None):
    """
    Create a choropleth map showing HDS rates from Cario freight quotes.
    
    Parameters:
    hds_data: DataFrame with columns ['customer_postcode', 'customer_locality', 'warehouse_postcode', 
                                    'warehouse_locality', 'freight_price', 'transit_time_days', 
                                    'carrier_id', 'freight_company']
    warehouse_data: Optional DataFrame with warehouse location data for markers
    """
    import geopandas as gpd
    import folium
    import pandas as pd
    
    # Make a copy of the data to avoid modifying the original DataFrame
    data = hds_data.copy()
    
    # Ensure that the price fields are numeric
    data['freight_price'] = pd.to_numeric(data['freight_price'], errors='coerce')
    data['transit_time_days'] = pd.to_numeric(data['transit_time_days'], errors='coerce')

    # Clip freight prices at $400 maximum
    data['freight_price'] = data['freight_price'].clip(upper=400)
    
    # Format the postcode field (and ensure string type)
    data['customer_postcode'] = data['customer_postcode'].astype(str).str.zfill(4)
    
    # Load and prepare the boundary GeoDataFrame
    boundary = gpd.read_file("boundary.json")
    boundary['POA_NAME'] = boundary['POA_NAME'].str[:4]
    boundary = boundary[boundary['geometry'].notnull()]
    
    # Merge boundary with the data based on the postcode
    hds_geometry_df = boundary.merge(data, left_on='POA_NAME', right_on='customer_postcode')
    
    # Create a base map centered over Australia
    hds_rate_map = folium.Map(location=[-24.15, 133.25], zoom_start=3)
    
    # Add a Choropleth layer for HDS Freight Price
    price_choropleth = folium.Choropleth(
        geo_data=boundary,
        data=data,
        bins=8,
        columns=["customer_postcode", "freight_price"],
        key_on="feature.properties.POA_NAME",
        fill_color="OrRd",
        nan_fill_color="lightgray",
        fill_opacity=0.7,
        line_opacity=0.2,
        name='Freight Price',
        highlight=True,
        legend_name="Freight Price ($AUD)",
    ).add_to(hds_rate_map)
    
    # Add a Choropleth layer for Transit Time
    transit_choropleth = folium.Choropleth(
        geo_data=boundary,
        data=data,
        bins=6,
        columns=["customer_postcode", "transit_time_days"],
        key_on="feature.properties.POA_NAME",
        fill_color="YlGnBu",
        nan_fill_color="white",
        fill_opacity=0.7,
        line_opacity=0.2,
        name='Transit Time',
        highlight=True,
        legend_name="Transit Time (Days)"
    ).add_to(hds_rate_map)
    
    # Define style and highlight functions for GeoJson layer
    style_function = lambda x: {
        'fillColor': '#ffffff',
        'color': '#000000',
        'fillOpacity': 0.01,
        'weight': 0.01
    }
    highlight_function = lambda x: {
        'fillColor': '#ffffff',
        'color': '#000000',
        'fillOpacity': 0.01,
        'weight': 0.01
    }
    
    # Add a GeoJson layer for detailed HDS info with tooltip
    hds_info_layer = folium.features.GeoJson(
        hds_geometry_df,
        style_function=style_function,
        control=True,
        name='HDS Details',
        highlight_function=highlight_function,
        tooltip=folium.features.GeoJsonTooltip(
            fields=["customer_postcode", "customer_locality", "freight_price", 
                   "transit_time_days", "freight_company", "warehouse_locality"],
            aliases=["Postcode: ", "Locality: ", "Price: $", 
                    "Transit Days: ", "Carrier: ", "From Warehouse: "],
            style=("background-color: white; color: #333333; font-family: arial; "
                   "font-size: 12px; padding: 10px;")
        )
    )
    hds_rate_map.add_child(hds_info_layer)
    
    # Add warehouse markers if warehouse data is provided
    if warehouse_data is not None:
        wh_data = warehouse_data.copy()
        
        # Group warehouses by price ranges for different marker colors
        if 'freight_price' in wh_data.columns:
            wh_data['freight_price'] = pd.to_numeric(wh_data['freight_price'], errors='coerce')
            
            # Warehouses with cheap rates (under $150)
            wh_cheap = wh_data[wh_data["freight_price"] < 150].copy().reset_index(drop=True)
            cheap_group = folium.FeatureGroup(name='Warehouses < $150')
            for i in range(len(wh_cheap)):
                if pd.notna(wh_cheap.iloc[i].get('warehouse_lat')) and pd.notna(wh_cheap.iloc[i].get('warehouse_long')):
                    cheap_group.add_child(folium.Marker(
                        location=[wh_cheap.iloc[i]['warehouse_lat'], wh_cheap.iloc[i]['warehouse_long']],
                        icon=folium.Icon(color='green', icon='info-sign'),
                        tooltip=f"{wh_cheap.iloc[i].get('warehouse_locality', 'Unknown')} | ${wh_cheap.iloc[i].get('freight_price', 'N/A')}"
                    ))
            
            # Warehouses with expensive rates ($150+)
            wh_expensive = wh_data[wh_data["freight_price"] >= 150].copy().reset_index(drop=True)
            expensive_group = folium.FeatureGroup(name='Warehouses ≥ $150')
            for i in range(len(wh_expensive)):
                if pd.notna(wh_expensive.iloc[i].get('warehouse_lat')) and pd.notna(wh_expensive.iloc[i].get('warehouse_long')):
                    expensive_group.add_child(folium.Marker(
                        location=[wh_expensive.iloc[i]['warehouse_lat'], wh_expensive.iloc[i]['warehouse_long']],
                        icon=folium.Icon(color='red', icon='info-sign'),
                        tooltip=f"{wh_expensive.iloc[i].get('warehouse_locality', 'Unknown')} | ${wh_expensive.iloc[i].get('freight_price', 'N/A')}"
                    ))
            
            hds_rate_map.add_child(cheap_group)
            hds_rate_map.add_child(expensive_group)
        else:
            # Add generic warehouse markers if no price data
            wh_group = folium.FeatureGroup(name='Warehouses')
            for i, row in wh_data.iterrows():
                if pd.notna(row.get('warehouse_lat')) and pd.notna(row.get('warehouse_long')):
                    wh_group.add_child(folium.Marker(
                        location=[row['warehouse_lat'], row['warehouse_long']],
                        icon=folium.Icon(color='blue', icon='info-sign'),
                        tooltip=f"{row.get('warehouse_locality', 'Unknown Warehouse')}"
                    ))
            hds_rate_map.add_child(wh_group)
    
    # Add layer control
    folium.LayerControl().add_to(hds_rate_map)
    
    return hds_rate_map

# Usage example:
hds_map = plot_hds_rates_map(cheapest_per_postcode)
hds_map
# hds_map.save('hds_rates_map.html')